<a href="https://colab.research.google.com/github/hbhb0311/Study/blob/main/Pytorch/PART3-CNN/ImageFolder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 나만의 데이터셋 만들기

In [22]:
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [23]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [24]:
# 사이즈 줄이기
trans = transforms.Compose([
            transforms.Resize([64, 64])
])
train_data = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/pytorch/image', transform = trans)

In [28]:
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)

    # imshow(data)
    # print(label)
    # break

    if label == 0:
        data.save('/content/drive/MyDrive/pytorch/train_data/sh/%d_%d.jpeg' %(num, label))
    else:
        data.save('/content/drive/MyDrive/pytorch/train_data/flower/%d_%d.jpeg' %(num, label))

0 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722765810> 0
1 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722765BD0> 0
2 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722765890> 0
3 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722765F10> 0
4 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722765C10> 0
5 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722769B10> 0
6 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722787290> 0
7 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722787310> 0
8 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722787450> 0
9 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722787250> 0
10 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD7227873D0> 0
11 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD7227875D0> 0
12 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722787090> 0
13 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722787350> 0
14 <PIL.Image.Image image mode=RGB size=64x64 at 0x7FD722787210> 0
15 <P

# ImageFolder2

In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [74]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [75]:
# 사이즈 줄이기
trans = transforms.Compose([
            transforms.ToTensor()
])
train_data = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/pytorch/train_data', transform = trans)

In [76]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers = 2)

In [77]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size = 5, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size = 5, stride = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Linear(16*13*13, 120),
            nn.ReLU(),
            nn.Linear(120, 2)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)

        return out


In [78]:
# test
net = CNN().to(device)
test_input = (torch.Tensor(3, 3, 64, 64)).to(device)
test_out = net(test_input)

# 결과
# torch.Size([3, 6, 30, 30])
# torch.Size([3, 16, 13, 13])
# torch.Size([3, 2704])

In [79]:
optimizer = optim.Adam(net.parameters(), lr = 0.001)
loss_func = nn.CrossEntropyLoss().to(device)

In [80]:
total_batch = len(data_loader)

epochs = 10
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch

    print('Epoch {} cost = {}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

Epoch 1 cost = 0.6942778825759888
Epoch 2 cost = 0.6756244897842407
Epoch 3 cost = 0.6154952645301819
Epoch 4 cost = 0.6023144721984863
Epoch 5 cost = 0.48735126852989197
Epoch 6 cost = 0.32095572352409363
Epoch 7 cost = 0.24504229426383972
Epoch 8 cost = 0.4273144006729126
Epoch 9 cost = 0.2260907143354416
Epoch 10 cost = 0.2162034511566162
Learning Finished!


## 모델 저장하기

In [84]:
torch.save(net.state_dict(), '/content/drive/MyDrive/pytorch/model/model.pth')

## 모델 불러오기

In [86]:
new_net = CNN().to(device)

In [87]:
new_net.load_state_dict(torch.load('/content/drive/MyDrive/pytorch/model/model.pth')) 

<All keys matched successfully>

In [88]:
print(net.layer1[0])
print(new_net.layer1[0])

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))


# 테스트

In [97]:
trans = torchvision.transforms.Compose([
                transforms.Resize([64, 64]),
                transforms.ToTensor()
])

test_data = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/pytorch/test_data', transform = trans)

In [98]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [99]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, labels = data
        imgs, labels = imgs.to(device), labels.to(device)

        prediction = net(imgs)
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy : ', accuracy.item())

Accuracy :  0.550000011920929
